In [2]:
# DRO actual data 

In [3]:
IN_COLAB = False
REMOUNT = False

In [4]:
# RUN IN COLAB ONLY: mount Google drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [5]:
# RUN IN COLAB ONLY: unmount, remount Google drive - if required
if REMOUNT:
    !fusermount -u /content/drive 2>/dev/null || true
    !rm -rf /content/drive
    from google.colab import auth
    auth.authenticate_user()
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [6]:
# RUN IN COLAB ONLY: force-fetch latest files from GitHub (overwrite local copies)
if IN_COLAB:
    !rm -rf /content/SLDS
    !git clone --depth=1 https://github.com/chrismader1/SLDS.git /content/SLDS
    import sys
    sys.path.append("/content/SLDS")

In [7]:
# RUN IN COLAB ONLY: prevent timeouts
if IN_COLAB:
    from IPython.display import Javascript, display
    display(Javascript("""
    (function keepAlive(){
      function clickConnect(){
        // Try shadow-root button (newer Colab)
        const el = document.querySelector('colab-connect-button');
        if (el && el.shadowRoot){
          const btn = el.shadowRoot.querySelector('#connect');
          if (btn){ btn.click(); console.log('keepAlive: clicked shadow connect'); return; }
        }
        // Fallbacks
        const btn2 = document.querySelector('#connect, button#connect');
        if (btn2){ btn2.click(); console.log('keepAlive: clicked #connect'); return; }
        console.log('keepAlive: connect button not found');
      }
      setInterval(clickConnect, 60 * 1000);
      console.log('keepAlive: armed');
    })();
    """))

In [8]:
# RUN IN COLAB ONLY: clear cache
if IN_COLAB:
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_results.csv"
    !rm -f "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/gridsearch_segments.csv"
    !echo "Reset done."

In [9]:
if IN_COLAB:
    !pip -q install -U "cupy-cuda12x>=13.0.0"
    !pip -q install -U cvxpy clarabel

In [10]:
if IN_COLAB:
    import cupy as cp
    assert cp.cuda.runtime.getDeviceCount() > 0
    print(cp.cuda.runtime.getDeviceProperties(0)['name'])

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools, warnings, os
from dro import *

# import warnings
# warnings.filterwarnings("ignore", message=r"Argument (sub|subj) .* Incorrect array format causing data to be copied")
# pd.set_option('future.no_silent_downcasting', True)

GPU=False


In [12]:
# paths

res = "gridsearch_results1"
par = "gridsearch_segments1"

COLAB_PATHS = {
    "data_excel": "/content/drive/MyDrive/Colab Notebooks/SLDS/Data/bbg_data.xlsx",  # Google Drive
    "results_csv": f"/content/drive/MyDrive/Colab Notebooks/SLDS/Out/{res}.csv",            # Google Drive
    "segments_parquet": f"/content/drive/MyDrive/Colab Notebooks/SLDS/Out/{par}.parquet",  # Google Drive
    "dro_pickle": "/content/drive/MyDrive/Colab Notebooks/SLDS/Out/dro_run.pkl.gz"
}

LOCAL_PATHS = {
    "data_excel": "/Users/chrismader/Python/SLDS/Data/bbg_data.xlsx",
    "results_csv": f"/Users/chrismader/Python/SLDS/Output/{res}.csv",
    "segments_parquet": f"/Users/chrismader/Python/SLDS/Output/{par}.parquet",
    "dro_pickle": "/Users/chrismader/Python/SLDS/Output/dro_run.pkl.gz"
}

In [13]:
# -------------------------
# CONFIG
# -------------------------

CONFIG = {
    # rSLDS model preferences
    "prefer_configs": [
        # {'config': '[g,v,h]',        'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[g,v]',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[y,g,v,h]',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': '[y,h]',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': '[y]',            'n_regimes': 3, 'dim_latent': 1},
        # {'config': 'factor1',        'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'factor1_vix',    'n_regimes': 3, 'dim_latent': 3},
        {'config': 'factor2_ff3',    'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'factor2_ff3mom', 'n_regimes': 3, 'dim_latent': 4},
        # {'config': 'factor2_ff5',    'n_regimes': 3, 'dim_latent': 5},
        # {'config': 'factor2_ff5mom', 'n_regimes': 3, 'dim_latent': 6},
        # {'config': 'fund1',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund1_vix',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'fund2',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund2_vix',      'n_regimes': 3, 'dim_latent': 3},
        # {'config': 'fund3',          'n_regimes': 3, 'dim_latent': 2},
        # {'config': 'fund3_vix',      'n_regimes': 3, 'dim_latent': 3},
    ],

    # Optimizer
    "seed": None,
    "start_dt": None,
    "end_dt": None,
    "min_assets": 3,
    "GLOBAL": {"risk_budget": 0.30, "risk_free_rate": 0.0, "epsilon_sigma": 1e-6},
    "delta_name": "bootstrap_gaussian",

    # Delta defaults (flat under CONFIG)
    "delta_defaults": {
        "kappa_l2":           {"delta_method": "kappa_l2",           "kappa": 1.0},
        "kappa_rate":         {"delta_method": "kappa_rate",         "kappa": 1.0},
        "bound_ek":           {"delta_method": "bound_ek",           "alpha": 0.05, "c1": 3.0, "c2": 1.0, "a": 2.0},
        "bootstrap_np":       {"delta_method": "bootstrap_np",       "alpha": 0.05, "B": 100, "seed": 0},
        "bootstrap_gaussian": {"delta_method": "bootstrap_gaussian", "alpha": 0.05, "B": 100, "seed": 0},
    },}

PATHS = COLAB_PATHS if IN_COLAB else LOCAL_PATHS
for k, v in PATHS.items(): 
    CONFIG[k] = v


In [14]:
# -------------------------
# Execute
# -------------------------

# res = pd.read_excel(CONFIG["data_excel"])
# securities = list(res["Ticker"].unique())
# securities = ['NVDA', 'AAPL', 'AVGO', 'GOOGL', 'ORCL', 'IBM', 'CSCO', 'CRM']
securities = None

out = dro_pipeline(securities, CONFIG, verbose=True)


[AAPL] raw segments: [0, 1308, 1569, 1831, 2092, 2353, 2613, 2734, 2735, 2833, 2834, 2835, 2875, 2893, 2923, 2957, 2967, 3079, 3105, 3137, 3400, 3463, 3464, 3521]
[ACN] raw segments: [0, 1308, 1569, 1831, 2092, 2353, 2367, 2370, 2388, 2389, 2391, 2442, 2443, 2587, 2588, 2613, 2834, 2835, 2893, 2897, 2919, 2923, 2958, 2964, 3080, 3096, 3137, 3400, 3463, 3464, 3521]
[ADBE] raw segments: [0, 1308, 1569, 1831, 2353, 2613, 3137, 3209, 3255, 3273, 3283, 3284, 3287, 3383, 3400, 3463, 3464, 3521]
[AMAT] raw segments: [0, 1308, 1831, 2125, 2127, 2128, 2130, 2135, 2137, 2138, 2141, 2144, 2147, 2148, 2149, 2155, 2156, 2166, 2167, 2875, 3137, 3272, 3274, 3277, 3279, 3285, 3287, 3308, 3352, 3355, 3382, 3384, 3521]
[AMD] raw segments: [0, 1308, 1569, 2092, 2140, 2141, 2148, 2150, 2353, 2613, 3209, 3273, 3283, 3284, 3285, 3384, 3400, 3463, 3464, 3521]
[AMZN] raw segments: [0, 1569, 1831, 2135, 2137, 2138, 2142, 2367, 2372, 2388, 2391, 2442, 2445, 2490, 2492, 2535, 2537, 2589, 2613, 2875, 3137, 3209, 

/opt/anaconda3/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


[t=1569 | 2018-01-05] active: ['AAPL', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'BKNG', 'COST', 'CRM', 'CSCO', 'DIS', 'GOOGL', 'HD', 'IBM', 'INTU', 'KO', 'MCD', 'META', 'MSFT', 'NFLX', 'NOW', 'NVDA', 'ORCL', 'PG', 'PM', 'QCOM', 'T', 'TSLA', 'TXN', 'VZ', 'WMT']


/opt/anaconda3/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


[t=1831 | 2019-01-08] active: ['AAPL', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'BKNG', 'COST', 'CRM', 'CSCO', 'DIS', 'GOOGL', 'HD', 'IBM', 'INTU', 'KO', 'MCD', 'META', 'MSFT', 'NFLX', 'NOW', 'NVDA', 'ORCL', 'PG', 'PM', 'QCOM', 'T', 'TSLA', 'TXN', 'VZ', 'WMT']


...

[t=3465 | 2025-04-14] active: ['AAPL', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'BKNG', 'COST', 'CRM', 'CSCO', 'DIS', 'GOOGL', 'HD', 'IBM', 'INTU', 'KO', 'MCD', 'META', 'MSFT', 'NFLX', 'NOW', 'NVDA', 'ORCL', 'PG', 'PLTR', 'PM', 'QCOM', 'T', 'TSLA', 'TXN', 'VZ', 'WMT']


/opt/anaconda3/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


[t=3499 | 2025-05-30] active: ['AAPL', 'ACN', 'ADBE', 'AMAT', 'AMD', 'AMZN', 'AVGO', 'BKNG', 'COST', 'CRM', 'CSCO', 'DIS', 'GOOGL', 'HD', 'IBM', 'INTU', 'KO', 'MCD', 'META', 'MSFT', 'NFLX', 'NOW', 'NVDA', 'ORCL', 'PG', 'PLTR', 'PM', 'QCOM', 'T', 'TSLA', 'TXN', 'VZ', 'WMT']

[MVO]    MVO baseline summary:
 mu_ann                0.8495
sigma_ann             0.7019
sharpe_ann            1.2265
vol_breach            0.4019
max_drawdown         -0.7918
gross_exp             3.3480
gap                   0.0000
kappa                 0.0000
delta                 0.0000
avg_holding_per    1239.0000
dtype: float64

[DRO]    Static DRO summary:
 mu_ann                0.8495
sigma_ann             0.7019
sharpe_ann            1.2265
vol_breach            0.4019
max_drawdown         -0.7918
gross_exp             3.3480
gap                   0.0000
kappa                    NaN
delta                 0.0000
avg_holding_per    1239.0000
dtype: float64

[RegDRO] Regime-DRO summary:
 mu_ann               

/opt/anaconda3/lib/python3.12/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


  k=137: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=138: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=139: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=140: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=141: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=142: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  k=143: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan n